# EXAMPLE 3
## Explaining the experiment
In this example, we are investigating a 128kx8 SRAM that was exposed to radiation in static tests with different bias voltages and with the same pattern, 0x55. Several irradiations were done yielding the CSV files present in this folder. 

According to the manufacturer, MBUs are impossible due to the presence of interleaving. Besides, the technology was so old that it was very difficult to induce MCUs in the memory with the conditions of the experiment.

## Loading packages
The very first thing we must do is to load the packages required to load files (_DelimitedFiles_) as well as the LELAPE module. I suppose you have installed both. Load is done with:

In [ ]:
### Optional
push!(LOAD_PATH, "PATH_TO_FOLDER_WITH_LELAPE.jl");
#if the following cell is not successfully executed, run this one with the details of your system.

In [1]:
using DelimitedFiles, LELAPE

┌ Info: Precompiling LELAPE [top-level]
└ @ Base loading.jl:1317


## Defining variables
Previous paragraph allows us to define several variables for checking the tests:

* Word width : 8 bits
* Memory size in words: 128k is just 2^17.
* In SRAMs, it seems more likely to succeed the XOR operation.
* Tests were static. No information about cycles is necessary.

Ok, let us use this information to set these variables:

In [2]:
LA = 2^17 # Memory size in words
WordWidth = 8 # Selfexplaining.
Operation = "XOR" # Only "XOR" or "POS" are allowed.
KeepCycles = false # This is a Bool variable and only true false are accepted.

false

## Loading data
Results are stored in three different files following the required format: 
* CSV files 
* Every row is formed as WORD ADDRESS, READ VALUE, PATTERN, CYCLE.
Besides, the first row contains column heading (must be skipped), separators are commas and EOL character is the standard. 

We will use the _readdlm_ function provided by the _DelimitedFiles_ package to load the first CSV file and to store everything in the new variable, DATA. Finally, it is important to indicate that DATA must be an array of UInt32 numbers. 

In [89]:
DATA10 = readdlm("ExampleSRAM10.csv", ',', UInt32, '\n', skipstart=1)

902×4 Matrix{UInt32}:
 0x000000eb  0x00000057  0x00000055  0x00000001
 0x0000014e  0x00000051  0x00000055  0x00000001
 0x0000019e  0x0000005d  0x00000055  0x00000001
 0x000001d6  0x00000057  0x00000055  0x00000001
 0x000001dc  0x00000075  0x00000055  0x00000001
 0x00000286  0x00000054  0x00000055  0x00000001
 0x000002ac  0x00000054  0x00000055  0x00000001
 0x000003a5  0x00000054  0x00000055  0x00000001
 0x00000524  0x00000051  0x00000055  0x00000001
 0x00000538  0x0000005d  0x00000055  0x00000001
 0x00000582  0x00000057  0x00000055  0x00000001
 0x000005a5  0x0000005d  0x00000055  0x00000001
 0x000005d6  0x00000075  0x00000055  0x00000001
          ⋮                          
 0x0001f525  0x0000005d  0x00000055  0x00000001
 0x0001f560  0x0000005d  0x00000055  0x00000001
 0x0001f649  0x00000045  0x00000055  0x00000001
 0x0001f986  0x00000075  0x00000055  0x00000001
 0x0001f9c8  0x000000d5  0x00000055  0x00000001
 0x0001fa44  0x00000075  0x00000055  0x00000001
 0x0001fab9  0x00000054  0x0

Good!! If you have correctly proceeded, a 902x4 unsigned integer matrix is loaded. Last column is useless and could have been omitted. At any rate, the software will solve the problem. Now, let us analyze the DATA10 set. First of all, the MBUs.

## Looking for MBUs
This analyisis is quite simple. We will call the _CheckMBUs_ functionthat returns the MBUs present in DATA.Input arguments are the second and third columns, and the wordwidth.

This function returns two vectors. The first one indicates in position _k_ the number of bitflips observed in the _kth_ word. The second one is a vector of vectors and contains more detailed information: not only the number of bitflips per word but the position of the flipped bit (0 = LSB, WordWidth-1 = MSB). 

In [48]:
MBUSize, MBU_bit_pos = CheckMBUs(DATA10[:,2], DATA10[:,3], WordWidth)

([1, 1, 1, 1, 1, 1, 1, 1, 1, 1  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 1], Any[[5], [4], [2], [4], [1], [6], [6], [1], [1], [2]  …  [5], [7], [1], [4], [5], [5], [7], [2], [1], [1]])

The following loop will show how many MBUs per number of flipped bits were observed:

In [49]:
for size = 1: WordWidth
    println("$size-bit MBUs: ", length(findall(MBUSize.==size)))
end

1-bit MBUs: 571
2-bit MBUs: 3
3-bit MBUs: 0
4-bit MBUs: 0
5-bit MBUs: 0
6-bit MBUs: 0
7-bit MBUs: 0
8-bit MBUs: 0


__WHAT???? THREE 2-BIT MCUS?__ This is nonsense since this is physically impossible. Perhaps the problem comes from the fact that the memory was overirradiated. Let us calculate the expected number of false 2-bit MBUs...

In [50]:
NF2BITMBU = NF2BitMCUs(DATA10, LA, "MBU", WordWidth, WordWidth)
println("We expected $NF2BITMBU false 2-bit MBUs in this  experiment.")

We expected 1.0978288650512695 false 2-bit MBUs in this  experiment.


Therefore, we expected in average 2.71... false 2-bit MBUs and observed 3. This makes sense.

## Looking for MCUs
As modern memories are interleaved, it is not worth investigating MBUs but MCUs. Now, the system will combine addresses in all the possible pairs and operate them to create a DV set. If there were no MCUs, their characteristics are known. 

In particular, we can state that if the expected number of elements repeated _k_ times in this set is lower than a very low positive number, it is impossible to observe this number of repetitions unless the Only SBU assumption fails. We will define this threshold as 0.001 (default, 0.05). 


Although without a solid theoretical background, it seems that using pseudoaddress instead of word address provides better results.

Some experiments seem to show that if an element with very few number of 1s in binary format is too often repeated, it is indicative of the presence of MCUs. This is the Trace Rule and, in our analyisis, we want to keep all those too often repeated elements such that contain 2 ones or less in binary format.

Finally, perhaps we know that MCUs will not very large. For example, we may guess that MCUs with more than 20 bitflips are totally rejected. Therefore, to help the software and to avoid running out of memory, we will say the program _"Don't be silly and do not expect events larger than 20!!"_ If somehow this idea was wrong, we can change this value again and repeat the calculations.

In [51]:
ϵ = 0.001   # If the expected number of elements repeated k times is lower than ϵ, 
            # we can afirm that this is virtually impossible.
UsePseudoAddress = true
TraceRuleLength = 2
LargestMCUSize = 20

20

Time to test!!! We will call the function. Deppending on the set size or even if this is your first test, it will take you more or less time (Don't get up from your chair, though!!!!)

The following instruction will look for:
1. Values that pass the self-consistency test (C10_SCY)
2. Values found after inspecting MCUs derived from self-consistency-test (C10_MCU).
3. Values with less than or equal to _TraceRuleLength_ 1s in binary format that appear too often in the DV set (C10_TRC).
4. Values that, after combining in pairs the union of all the previous three sets and applying the operation and that appear too many times within the DV set (C10_SHF).

The first column of each matrix are the possible values and the second one the times it appeared.

In [88]:
C10_SCY, C10_MCU, C10_TRC, C10_SHF = DetectAnomalies_FullCheck(DATA10, WordWidth, LA, Operation, TraceRuleLength, UsePseudoAddress, KeepCycles, ϵ, LargestMCUSize)


(UInt32[0x00000010 0x0000000e], Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2), Matrix{UInt32}(undef, 0, 2))

### What has happened? 
All the returned matrices are void. The reason is that there are not MCUs, only SBUs, or at least are so scarce that cannot be distinguished from statistical deviations. Now, we can conclude the analysis since all the bitflips are SBUs.

## Further analysis
You can find in this folder other actual data got from the same memory. Analyze them and, if you wish. For your information, there are at least some files that yield some positive results.